## PostgreSQL in Python

Pre-requisties:
- PostgreSQL installed on your machine

 The [psycopg2](https://pypi.org/project/psycopg2/) package allows SQL querying in a Python environment. Start by importing the package and connecting to the [default postgres database](https://stackoverflow.com/questions/2370525/default-database-named-postgres-on-postgresql-server). This is a necessary step before creating any new databases.

In [1]:
import psycopg2

In [2]:
# Connect to the default database
conn = psycopg2.connect(
    dbname = "postgres",
    user = "postgres",
    password = "****" # Use password provided at postgresql installation
)

We can now use this connection to get a [cursor](https://www.psycopg.org/docs/cursor.html#:~:text=class%20cursor,session%20wrapped%20by%20the%20connection.) which allows for query execution. For our queries to occur, however, we must commit them. This can be done by either running ```conn.commit()``` everytime we execute a query or by setting up **autocommit**. Autocommit closes the transaction when an action occurs so that no more actions can take place. If you do not set up autocommit you will have to call ```conn.commit()``` at the end of each command which can be useful in certain situations (e.g. testing for an outcome).

In [3]:
# Get cursor
cur = conn.cursor()

# Set up autocommit
conn.set_session(autocommit = True)

Queries can now be executed using the ```.execute()``` method which takes in a string of SQL syntax, and should be committed automatically. Let's try to create a new database to connect to.

In [4]:
# Creating a new database
cur.execute("CREATE DATABASE test_db;")

Before connecting to the new database, we must first close the existing connection with the default database. Then we can get a new cursor and set up autocommit.

In [5]:
# Close existing connection
conn.close()

# Connect to new database
conn = psycopg2.connect(
    dbname = "test_db",
    user = "postgres",
    password = "qweasdzxc21"
)

cur = conn.cursor()
conn.set_session(autocommit = True)

Let's try creating a table with some columns and check if we were successful.

In [6]:
# Create table music_library
cur.execute("CREATE TABLE IF NOT EXISTS music_library (album_name varchar, artist_name varchar, year int);")

# Check if table was created by returning number of rows
cur.execute("SELECT COUNT(*) FROM music_library;")
print(cur.fetchall())

[(0,)]


The ```.fetchall()``` method returns all (remaining) rows of a query result as a list of tuples. The result was zero which is correct since no rows were insterted into the table. Let's see how to insert a couple of rows and validate that they were inserted.

In [7]:
# Insert row 1
cur.execute("INSERT INTO music_library (album_name, artist_name, year) \
    VALUES (%s, %s, %s)",
    ("Let It Be", "The Beatles", 1970)    
)

# Insert row 2 
cur.execute(
    "INSERT INTO music_library (album_name, artist_name, year) \
        VALUES (%s, %s, %s)", 
        ("Rubber Soul", "The Beatles", 1965)
)

*Note*: If we run the insert statements more than once, duplicate data will be created. <br><br>
As we are not executing queries in a Postgres shell, we will need a while loop to print the results with Python.

In [8]:
cur.execute("SELECT * FROM music_library;")

row = cur.fetchone() # gets first row
while row:
    print(row)
    row = cur.fetchone() # gets next row

('Let It Be', 'The Beatles', 1970)
('Rubber Soul', 'The Beatles', 1965)


Finally, close the connection to the database.

In [9]:
cur.close()
conn.close()

Create a new user? [Click here](https://phoenixnap.com/kb/postgres-create-user) for instructions.